In [1]:
# !rm -rf /kaggle/working/*
# !conda install -y gdown
# print('hi')
!git clone https://github.com/mattjhayes3/cs234-project.git
%cd /kaggle/working/cs234-project
!pip install wandb
import wandb

wandb.login(key="KEY")
# wandb.init()

from huggingface_hub import notebook_login,login

# notebook_login("KEY")
login("KEY")

# !gdown --id 1TTg8s_dj60EKl4No2unSvYmMyMopPVJ6
# !gdown --id 1Z7HvAokBQu65jew4ou2DjOYRi23OrJdr
!mkdir results
!pip install datasets>=1.17.0 torch>=1.4.0 tqdm transformers accelerate peft>=0.3.0 tyro>=0.5.7
!pip install git+https://github.com/mattjhayes3/trl.git

Cloning into 'cs234-project'...
remote: Enumerating objects: 226, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 226 (delta 1), reused 3 (delta 1), pack-reused 220
Receiving objects: 100% (226/226), 48.42 MiB | 25.35 MiB/s, done.
Resolving deltas: 100% (108/108), done.
Updating files: 100% (99/99), done.
/kaggle/working/cs234-project


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful
  Cloning https://github.com/mattjhayes3/trl.git to /tmp/pip-req-build-u0kcb52b
  Running command git clone --filter=blob:none --quiet https://github.com/mattjhayes3/trl.git /tmp/pip-req-build-u0kcb52b
  Resolved https://github.com/mattjhayes3/trl.git to commit 2719f4cd3aea1c0d75ab12d6a2a7ba5697c99e95
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for trl: filename=trl-0.8.7.dev0-py3-none-any.whl size=209506 sha256=5034b89b2299184f5eaf66cd5075a7a3d59a6c2690fedf569e05f3dad2d24ed7
  Stored in directory: /tmp/pip-ephem-wheel-cache-q6dedlan/wheels/b5/f5/74/f982e27bdb1c23b205f8bfcaaed174cf3d2c06bd1a5f5926cc
Successfully built trl


In [ ]:
!pip install git+https://github.com/mattjhayes3/trl.git && python ppo.py --exp_name=roberta16-256_2_0 --log_with=wandb --init_kl_coef=2_0


In [ ]:
!pip install git+https://github.com/mattjhayes3/trl.git && python ppo.py --exp_name=roberta16-256_0_2 --log_with=wandb --init_kl_coef=0.2


In [ ]:
!pip install git+https://github.com/mattjhayes3/trl.git && python ppo.py --exp_name=roberta16-256_0_1 --log_with=wandb --init_kl_coef=0.1

In [ ]:
!pip install git+https://github.com/mattjhayes3/trl.git && python ppo.py --exp_name=roberta16-256_0_05 --log_with=wandb --init_kl_coef=0.05

In [ ]:
!pip install git+https://github.com/mattjhayes3/trl.git && python ppo.py --exp_name=eval --log_with=wandb --eval_model=DownwardSpiral33/gpt2-imdb-pos-roberta16-256_0_05-2024.05.20.06.39


In [ ]:
!pip install git+https://github.com/mattjhayes3/trl.git && python ppo.py --exp_name=roberta16-256_0_02 --log_with=wandb --init_kl_coef=0.02

In [ ]:
!pip install git+https://github.com/mattjhayes3/trl.git && python ppo.py --exp_name=roberta16-256_0_035 --log_with=wandb --init_kl_coef=0.035

In [3]:
!pip install trl
import torch
from tqdm import tqdm
import pandas as pd

tqdm.pandas()

from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

import torch.nn.functional as F
model_name = "lvwerra/gpt2-imdb"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token



def build_dataset(dataset_name="imdb", input_min_text_length=2, input_max_text_length=8):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.

    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    # load imdb with datasets
    ds = load_dataset(dataset_name, split="train")
    ds = ds.rename_columns({"text": "review"})
    ds = ds.filter(lambda x: len(x["review"]) > 200, batched=False)

    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["review"])[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds

dataset = build_dataset()


def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

# %cd /content/drive/MyDrive/cs234_project

tokenizer.pad_token = tokenizer.eos_token
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "max_new_tokens": 16,
    # "num_return_sequences": 4
}

config = PPOConfig(
    model_name="lvwerra/gpt2-imdb",
    exp_name='sft-generation',
    # model_name="edbeeching/gpt2-large-imdb",
    learning_rate=1.41e-5,
    # log_with="wandb",
    # init_kl_coef=0.02,
    init_kl_coef=1.0,
    # target=10,
    adap_kl_ctrl=False,
    # steps=1000,
    batch_size=512,
    mini_batch_size=512,
    # reward_model="sentiment-analysis:lvwerra/distilbert-imdb",
    reward_model="sentiment-analysis:siebert/sentiment-roberta-large-english",
)

model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
ppo_trainer = PPOTrainer(config, model, model, tokenizer, dataset=dataset, data_collator=collator)
device = ppo_trainer.accelerator.device

sent_kwargs = {"return_all_scores": True, "function_to_apply": "softmax", "batch_size": config.batch_size}
task, model_name = config.reward_model.split(":")
print("Load reward model", model_name)
sentiment_pipe = pipeline(task, model=model_name, device=device)

# Some tokenizers like GPT-2's don't have a padding token by default, so we set one here.
if sentiment_pipe.tokenizer.pad_token_id is None:
    sentiment_pipe.tokenizer.pad_token_id = tokenizer.pad_token_id

dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=config.batch_size,
    collate_fn=collator,
    shuffle=False,
    drop_last=False,
)

all_results = []
with torch.no_grad():
  for _epoch, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
      # if _epoch < len(dataloader)-1:
      #   print(batch['query'])
      #   continue
      # if  _epoch > 1:
      #     break
      query_tensors = batch["input_ids"]

      batch_results = {'query': batch['query']}
      for gen_num in range(8):
        # print('gen num', gen_num)
        # Get response from gpt2
        response_tensors= ppo_trainer.generate(
            query_tensors, return_prompt=False, generate_ref_response=False, batch_size=config.batch_size, **generation_kwargs
        )
        batch["response"] = tokenizer.batch_decode(response_tensors)
        # batch["ref_response"] = tokenizer.batch_decode(ref_response_tensors)

        # Compute sentiment score
        texts = [q + r for q, r in zip(batch["query"], batch["response"])]
        # print("texts", texts)
        pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
        batch_results[f'generation_{gen_num}'] = batch['response']
        batch_results[f'sentiment_{gen_num}'] = [output[1]["score"] for output in pipe_outputs]
      batch_df = pd.DataFrame(batch_results)
      # print(batch_df)
      all_results.append(batch_df)
pd.concat(all_results, axis=0).reset_index(drop=True).to_csv(f'sft_generations_16_token.csv')



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Load reward model siebert/sentiment-roberta-large-english


  0%|          | 0/49 [00:00<?, ?it/s]You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
  2%|▏         | 1/49 [00:18<14:25, 18.04s/it]--- Logging error ---
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 1100, in emit
    msg = self.format(record)
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 943, in format
    return fmt.format(record)
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 678, in format
    record.message = record.getMessage()
  File "

In [5]:
!python dpo.py  --output_dir=test   --dataset_name=DownwardSpiral33/gpt2-imdb-16-token-pref-pairs     --model_name_or_path=lvwerra/gpt2-imdb     --per_device_train_batch_size 128     --learning_rate 1e-3     --gradient_accumulation_steps 1     --logging_steps 10     --eval_steps 500     --warmup_steps 150     --report_to wandb     --logging_first_step --no_remove_unused_columns

  Cloning https://github.com/mattjhayes3/trl.git to /tmp/pip-req-build-jjghxu7t
  Running command git clone --filter=blob:none --quiet https://github.com/mattjhayes3/trl.git /tmp/pip-req-build-jjghxu7t
  Resolved https://github.com/mattjhayes3/trl.git to commit 314d80831b69b67a3da2034677bdf179c9da22e9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
2024-05-22 08:35:28.484112: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-22 08:35:28.484167: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-22 08:35:28.485591: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [6]:
!python ppo.py --exp_name=eval --log_with=wandb --eval_model=./test-2024.05.22.08.35


2024-05-22 09:41:56.860500: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-22 09:41:56.860569: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-22 09:41:56.862120: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
PPOConfig: PPOConfig(exp_name='eval', dry_run=False, eval_model='./test-2024.05.22.08.35', seed=0, log_with='wandb', task_name=None, model_name='lvwerra/gpt2-imdb', query_dataset='imdb', reward_model='sentiment-analysis:siebert/sentiment-roberta-large-english', remove_unused_columns=True, tracker_kwargs={'wandb': {'name': 'eval-2024.05.22.09.42'}}, accelerator_kw

In [3]:
!git pull --rebase && python dpo.py  --output_dir=test  --beta=0.2 --dataset_name=DownwardSpiral33/gpt2-imdb-16-token-pref-pairs     --model_name_or_path=lvwerra/gpt2-imdb     --per_device_train_batch_size 128     --learning_rate 1e-3     --gradient_accumulation_steps 1     --logging_steps 10     --eval_steps 250     --warmup_steps 150     --report_to wandb     --logging_first_step --no_remove_unused_columns

remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 8 (delta 2), reused 8 (delta 2), pack-reused 0
Unpacking objects: 100% (8/8), 3.45 KiB | 1.72 MiB/s, done.
From https://github.com/mattjhayes3/cs234-project
   8310e02..b635666  main       -> origin/main
Updating 8310e02..b635666
Fast-forward
 dpo.py            | 183 ++++++++++++++++++++++++++++++++++++++++++++++++++++++
 gen_pref_pairs.py |  41 ++++++++++++
 trl               |   2 +-
 wandb/latest-run  |   2 +-
 4 files changed, 226 insertions(+), 2 deletions(-)
 create mode 100644 dpo.py
 create mode 100644 gen_pref_pairs.py
Current branch main is up to date.
2024-05-22 10:35:30.564551: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-22 10:35:30.564643: E external/local_xla/xla/stream_executor/cuda/

Exception in thread OutRawRd-stderr:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/internal/sender.py", line 1231, in _output_raw_reader_thread
    self._output_raw_flush(stream)
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/internal/sender.py", line 1246, in _output_raw_flush
    self._output_raw_file.write(data.encode("utf-8"))
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/lib/filesystem.py", line 128, in write
    super().write(b"\n".join(ret) + b"\n")
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/lib/filesystem.py", line 95, in write
    self.f.flush()
OSError: [Errno 28] No space left on device


In [ ]:
####### begin small dpos  #####

In [3]:
! python dpo.py  --output_dir=dpo_2c2_div36-0_05  --beta=0.05 --dataset_name=/kaggle/working/cs234-project/pref_pairs_16_token_2_choose_2_div_36_tokenized     --model_name_or_path=lvwerra/gpt2-imdb     --per_device_train_batch_size 128  --per_device_eval_batch_size 128     --learning_rate 2e-6     --gradient_accumulation_steps 1  --warmup_steps 50     --report_to wandb     --logging_first_step --logging_strategy='steps' --logging_steps 50  --eval_steps=50 --no_remove_unused_columns --save_total_limit=4 --evaluation_strategy='steps' --lr_scheduler_type=constant_with_warmup --save_strategy='epoch' --num_train_epochs=20

2024-05-31 11:19:07.647176: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-31 11:19:07.647232: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-31 11:19:07.648999: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.

In [4]:
!python ppo.py --exp_name=eval --eval_model=./dpo_2c2_div36-0_05-2024.05.31.11.19/checkpoint-589
!python ppo.py --exp_name=eval --eval_model=./dpo_2c2_div36-0_05-2024.05.31.11.19/checkpoint-558
!python ppo.py --exp_name=eval --eval_model=./dpo_2c2_div36-0_05-2024.05.31.11.19/checkpoint-527

2024-05-31 11:38:55.194471: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-31 11:38:55.194528: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-31 11:38:55.196132: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
PPOConfig: PPOConfig(exp_name='eval', normalize_scores=False, dry_run=False, eval_model='./dpo_2c2_div36-0_05-2024.05.31.11.19/checkpoint-589', seed=0, log_with=None, task_name=None, model_name='lvwerra/gpt2-imdb', query_dataset='imdb', reward_model='sentiment-analysis:siebert/sentiment-roberta-large-english', remove_unused_columns=True, tracker_kwargs={'wandb': 

In [7]:
! python dpo.py  --output_dir=dpo_2c2_div36-0_2  --beta=0.2 --dataset_name=/kaggle/working/cs234-project/pref_pairs_16_token_2_choose_2_div_36_tokenized     --model_name_or_path=lvwerra/gpt2-imdb     --per_device_train_batch_size 128  --per_device_eval_batch_size 128     --learning_rate 2e-6     --gradient_accumulation_steps 1  --warmup_steps 50     --report_to wandb     --logging_first_step --logging_strategy='steps' --logging_steps 50  --eval_steps=50 --no_remove_unused_columns --save_total_limit=4 --evaluation_strategy='steps' --lr_scheduler_type=constant_with_warmup --save_strategy='epoch' --num_train_epochs=19

2024-05-31 11:55:41.862548: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-31 11:55:41.862611: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-31 11:55:41.864299: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.

In [11]:
!rm -rf */*/optimizer.pt

In [18]:
!wandb login --relogin && cat /kaggle/working/cs234-project/wandb/debug-cli.root.log

wandb: ERROR Find detailed error logs at: /kaggle/working/cs234-project/wandb/debug-cli.root.log
Error: Connection to wandb service failed: [Errno 111] Connection refused. 


In [19]:
!ping api.wandb.ai 

/bin/bash: ping: command not found


In [20]:
wandb.login(key="KEY")
wandb.init()

! git pull --rebase && python dpo.py  --output_dir=dpo_2c2_div36-0_2  --beta=0.2 --dataset_name=/kaggle/working/cs234-project/pref_pairs_16_token_2_choose_2_div_36_tokenized     --model_name_or_path=lvwerra/gpt2-imdb     --per_device_train_batch_size 128  --per_device_eval_batch_size 128     --learning_rate 2e-6     --gradient_accumulation_steps 1  --warmup_steps 50     --report_to wandb     --logging_first_step --logging_strategy='steps' --logging_steps 50  --eval_steps=50 --no_remove_unused_columns --save_total_limit=4 --evaluation_strategy='steps' --lr_scheduler_type=constant_with_warmup --save_strategy='epoch' --num_train_epochs=14

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/wandb_init.py", line 1181, in init
    run = wi.init()
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/wandb_init.py", line 725, in init
    backend.interface.publish_header()
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/interface/interface.py", line 104, in publish_header
    self._publish_header(header)
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/interface/interface_shared.py", line 338, in _publish_header
    self._publish(rec)
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/interface/interface_sock.py", line 51, in _publish
    self._sock_client.send_record_pu

Error: An unexpected error occurred